In [ ]:

# librerías
import json
import pandas as pd
import hashlib
import unicodedata

"""
Realizamos un ETL a los archivos extraídos de Overpass Turbo,
normalizando todos los nombres y generando IDs únicos.
"""

#Función para normalizar textos
def normalizar_texto(texto):
    texto = str(texto).lower().strip()
    texto = unicodedata.normalize('NFKD', texto).encode('ascii', 'ignore').decode('utf-8')
    return texto

# Generamos IDs únicos para los nombres de los lugares 
def generar_id(nombre):
    normalizado = normalizar_texto(nombre)
    return "NOM" + hashlib.md5(normalizado.encode()).hexdigest()[:6].upper()

# Cargamos el archivo JSON de Overpass Turbo
ruta_json_overpass = r'H:/git/proyecto grupal 2/Yelp-Gmaps-Proyecto-DS/data/raw/overpass-turbo.eu/export.json'
with open(ruta_json_overpass, 'r', encoding='utf-8') as f:
    data = json.load(f)

# 📦 Extraer nombres únicos
nombres_nuevos = []
for elemento in data.get('elements', []):
    tags = elemento.get('tags', {})
    nombre = tags.get('name')
    if nombre:
        nombres_nuevos.append(normalizar_texto(nombre))

#  Eliminamos los duplicados locales
nombres_nuevos = sorted(set(nombres_nuevos))

# 📂 Cargamos la dimensión existente y normalizar sus nombres
ruta_dim_existente = r'H:\git\proyecto grupal 2\Yelp-Gmaps-Proyecto-DS\data\processed\dim\dim_nombre_restaurante.csv'
df_dim = pd.read_csv(ruta_dim_existente)
df_dim['nombre'] = df_dim['nombre'].astype(str).apply(normalizar_texto)

# Determinamos los nombres que aún no están en la dimensión
nombres_existentes = set(df_dim['nombre'])
nombres_a_agregar = [n for n in nombres_nuevos if n not in nombres_existentes]

# Creamos un DataFrame con los nuevos registros
nuevos_registros = pd.DataFrame({
    'id_nombre': [generar_id(nombre) for nombre in nombres_a_agregar],
    'nombre': nombres_a_agregar
})

# Concatenamos lo nuevos registros con la dimensión existente
df_dim_actualizada = pd.concat([df_dim, nuevos_registros], ignore_index=True)
df_dim_actualizada = df_dim_actualizada.drop_duplicates(subset='nombre')

# Guardamos la dimensión actualizada
ruta_salida_dim = r'H:/git/proyecto grupal 2/Yelp-Gmaps-Proyecto-DS/data/processed/dim/dim_nombre_restaurante_actualizada.csv'
df_dim_actualizada.to_csv(ruta_salida_dim, index=False, encoding='utf-8-sig')

print(f" ¡Dimensión actualizada y normalizada correctamente!\n Guardada en: {ruta_salida_dim}")




 ¡Dimensión actualizada y normalizada correctamente!
 Guardada en: H:/git/proyecto grupal 2/Yelp-Gmaps-Proyecto-DS/data/processed/dim/dim_nombre_restaurante_actualizada.csv
